In [1]:
import json

import numpy as np
import pandas as pd
from money_parser import price_dec

pd.options.display.max_columns = None

In [2]:
def curr_to_float(v): return float(price_dec(v)) if v else v
def tf_to_10(v): return 1 if v == 't' else 0
def nulls_to_0(v): return float(0 if (pd.isnull(v) or v == '') else v)

converters_to_columns = [
    (curr_to_float, ['price', 'weekly_price', 'monthly_price']),
    (tf_to_10, ['instant_bookable', 'host_is_superhost']),
    (nulls_to_0, ['bedrooms', 'bathrooms'])
]

converters = {col: conv for conv, cols in converters_to_columns for col in cols}

df = pd.read_csv('data/listings.csv', index_col='id', converters=converters)

/home/abhivik/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
complaints_df = pd.read_csv('data/NYPD_Complaint_Data_Historic.csv', index_col='CMPLNT_NUM')

In [45]:
cleaned_complaints_df = complaints_df[~complaints_df['Latitude'].isnull()]

In [236]:
lats = pd.cut(cleaned_complaints_df['Latitude'], 30)
lons = pd.cut(cleaned_complaints_df['Longitude'], 30)

In [238]:
lats = lats.apply(lambda v: str(v.left) + ',' + str(v.right))
lons = lons.apply(lambda v: str(v.left) + ',' + str(v.right))

In [239]:
boxes = lats.str.cat(lons, sep=',')

In [240]:
cleaned_complaints_df['boxes'] = boxes

In [156]:
# def to_tuple_str(v1, v2): return '(' + str(v1) + ', ' + str(v2) + ')'
# lats = lats.apply(lambda v: '(' + to_tuple_str(v.left, v.right))
# lons = lons.apply(lambda v: to_tuple_str(v.left, v.right) + ')')
# cleaned_complaints_df['boxes'] = lats.str.cat(lons, sep=', ')

In [241]:
def convert_to_number(val):
    if (val == 'FELONY'):
        return 5
    elif (val == 'MISDEMEANOR'):
        return 1
    else:
        return 0.1

cleaned_complaints_df['LAW_CAT_CD_NUMBER'] = cleaned_complaints_df['LAW_CAT_CD'].apply(convert_to_number)

In [242]:
box_df = cleaned_complaints_df[['boxes', 'LAW_CAT_CD_NUMBER']].groupby('boxes').sum().copy()

In [243]:
box_df = box_df.reset_index()

In [244]:
box_df['lower_latitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[0]))
box_df['upper_latitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[1]))
box_df['lower_longitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[2]))
box_df['upper_longitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[3]))

In [245]:
box_df['lower_latitude'] = box_df['lower_latitude'].apply(lambda v: 0 if v == box_df['lower_latitude'].min() else v)
box_df['upper_latitude'] = box_df['upper_latitude'].apply(lambda v: (v+1) if v == box_df['upper_latitude'].max() else v)
box_df['lower_longitude'] = box_df['lower_longitude'].apply(lambda v: (v-1) if v == box_df['lower_longitude'].min() else v)
box_df['upper_longitude'] = box_df['upper_longitude'].apply(lambda v: 0 if v == box_df['upper_longitude'].max() else v)

In [247]:
df['crime_score'] = np.nan
done_count = 0
for index, row in df.iterrows():
    s1 = np.logical_and(row.latitude > box_df['lower_latitude'], row.latitude <= box_df['upper_latitude'])
    s2 = np.logical_and(row.longitude > box_df['lower_longitude'], row.longitude <= box_df['upper_longitude'])
    box = box_df[np.logical_and(s1, s2)]
    try:
        df.at[index, 'crime_score'] = box['LAW_CAT_CD_NUMBER'].tolist()[0]
    except Exception as e:
        display(index)
    done_count+=1
    if done_count % 1000 == 0:
        print('done ', done_count)

done  1000
done  2000
done  3000
done  4000
done  5000
done  6000
done  7000
done  8000
done  9000
done  10000
done  11000
done  12000
done  13000
done  14000
done  15000
done  16000
done  17000
done  18000
done  19000
done  20000
done  21000
done  22000
done  23000
done  24000
done  25000
done  26000
done  27000
done  28000
done  29000
done  30000
done  31000
done  32000
done  33000
done  34000
done  35000
done  36000
done  37000
done  38000
done  39000
done  40000
done  41000
done  42000
done  43000
done  44000
done  45000
done  46000
done  47000
done  48000
done  49000
done  50000


In [ ]:
# df['crime_measure'] = np.nan
# total_crimes = len(weird_df.index)

# done_count = 0
# for listing_index, listing_row in df.iterrows():
#     total_distance = 0
#     lat, lon = listing_row['latitude'], listing_row['longitude']
#     for index, row in weird_df.iterrows():
#         distance = ((row['Latitude']['mean'] - lat)**2 + (row['Longitude']['mean'] - lon)**2) * 10000000
#         total_distance += (row['LAW_CAT_CD_NUMBER', 'sum'] / (distance))
#     df.at[listing_index, 'crime_measure'] = total_distance
#     done_count+=1
#     if done_count % 1000 == 0:
#         print('done ', done_count)

In [255]:
df['crime_score'].to_csv('crime_score.csv')

/home/abhivik/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
